# How to create a log file for the GoPiGo test

## Log file format  
Lets consider a log file example:  

49173,gpg.mic,Start  
65117,gpg.mic,street_music  
82348,gpg.mic,gun_shot  
88866,gpg.find_cone,blue,bottle  
90647,gpg.mic,dog_bark  
95237,gpg.mic,children_playing  
126866,gpg.find_cone,red,book  
183293,gpg.find_cone,yellow,suitcase  
212194,gpg.find_cone,green,suitcase   
212528,gpg.mic,Finish  

Each row contains 3 or 4 comma separated fields:  
- time in milliseconds elapsed since the start of the main logger;  
- logger name  
- log message (one or two fields)

To create a log file we will use a package 'logging'.  
We should initiate the parent logger object named 'gpg' and two child loggers: 'gpg.mic' (for sound recognition thread) and 'gpg.find_cone' (for visual objects recognition).  
The parent logger 'gpg' will recieve the messages from all child loggers named 'gpg.*'

In the main process:

In [1]:
import logging
main_logger = logging.getLogger('gpg')
main_logger.setLevel(logging.DEBUG)
fname = 'gopigo.log' # Any name for the log file

# Create the FileHandler object. This is required!
fh = logging.FileHandler(fname, mode='w')
fh.setLevel(logging.INFO)  # Will write to the log file the messages with level >= logging.INFO

# The following row is strongly recommended for the GoPiGo Test!
fh_formatter = logging.Formatter('%(relativeCreated)d,%(name)s,%(message)s')
fh.setFormatter(fh_formatter)
main_logger.addHandler(fh)
    
# The StreamHandler is optional for you. Use it just to debug your program code
sh = logging.StreamHandler()
sh_formatter = logging.Formatter('%(relativeCreated)8d %(name)s %(levelname)s %(message)s')
sh.setLevel(logging.DEBUG)
sh.setFormatter(sh_formatter)
main_logger.addHandler(sh)
main_logger.debug('Logger started') # This debug message will be handled only by StreamHendler


    1974 gpg DEBUG Logger started


Logger for the sound recognition:

In [2]:
mic_logger = logging.getLogger('gpg.mic')

# ...
# After GoPiGo has recongized the start signal (1760 Hz)
# It is important to write the 'Start' message immediately after the signal!
mic_logger.info('Start')

# ...
# GoPiGo has detected some sound, for example 'street_music'
class_name = 'street_music'
mic_logger.info(class_name)

# ...
# Sound recognition thread stops to listen the microphone because GoPiGo reached the finish point
mic_logger.info('Finish')

    1983 gpg.mic INFO Start
    1984 gpg.mic INFO street_music
    1985 gpg.mic INFO Finish


Visual objects recognition:

In [3]:
visual_logger = logging.getLogger('gpg.find_cone')

# ...
# After GoPiGo has detected some object (let be a bottle) near a colored (let be yellow) cone
color_name = 'yellow'
class_name = 'bottle'
visual_logger.info('%s,%s' % (color_name, class_name))

    1992 gpg.find_cone INFO yellow,bottle


After running all cells you can find a log file 'gopigo.log' in the current directory.

# Send the log file to the server and get the score

To send your log file to the server and get the score call `connection` from "robo_client.py". You also need "robotics_pb2.py".

`connection(HOST, PORT, login, password, split_id, filename)`
- HOST - 'datastream.ilykei.com'
- PORT - 30078
- login - your ilykey login
- password - your password for data stream assignments
- split id - should be 19
- filename - name of your log file


In [ ]:
    from robo_client import connection
    
    HOST, PORT = 'datastream.ilykei.com', 30078
    login = 'Your Login'
    password = 'Your Password for Data Stream Assignments'
    split_id = 19
    filename = 'gpg.log'
    connection(HOST, PORT, login, password, split_id, filename)    